# k-Nearest Neighbors & K-Means

_Welcome back! Today we’ll build two core ML tools that you’ll use again and again._

- **k-Nearest Neighbors (kNN)** — a simple, powerful **supervised** method
- **K-Means** — a fast, practical **unsupervised** clustering method

---

## What you'll learn

- kNN intuition and how prediction works
- Distances, feature scaling, and choosing **k**
- K-Means objective and the Lloyd’s algorithm loop
- How to pick **K** (elbow & silhouette)
- Minimal, copy-pasteable code for both

---

## Requirements

- Vectors & basic distance (e.g., Euclidean)
- Train/validation/test split
- Python + NumPy/Matplotlib (optionally scikit-learn)

---

## 1) kNN — the core idea (intuition)

It works on the idea that nearby points tend to share similar labels or values. The only real knob is **k**: small k focuses on very local patterns (can overfit to noise), while larger k smooths decisions (can miss fine detail). For classification, the class with the most support among the k neighbors is chosen; for regression, the neighbors’ values are averaged. Distance-weighting lets nearer neighbors count more than farther ones.

> **“Tell me who your neighbors are, and I’ll tell you who you are.”**

For a new point $x$:

1. Find the **k closest** training points.
2. **Classification:** majority vote of their labels.
3. **Regression:** average their target values.

```
              ●  ●      Class A
     x ?  →   ▲          Query
              ○  ○      Class B
```

`x` takes the class of whichever neighbors are more common (or closer, if weighted).

---

## 2) Distance & scaling (the make-or-break)

Common distances for vectors $x, y$:

- **Euclidean:** $\lVert x - y\rVert_2$ (default for geometry)
- **Manhattan:** $\lVert x - y\rVert_1$ (robust to outliers)
- **Cosine distance:** $1 - \dfrac{x^\top y}{\lVert x\rVert\,\lVert y\rVert}$ — “angle” difference (great for text/high-dim sparse)

**Euclidean** distance treats differences along each feature dimension equally, so features with larger scales can dominate if you don’t standardize. **Manhattan** distance is less sensitive to outliers and can work better with high-variance features. **Cosine** similarity ignores magnitude and compares only direction, which is useful for sparse/high‑dimensional data (e.g., text). Standardizing features (mean 0, std 1) before kNN keeps distances meaningful. If two classes are tied by count, distance‑weighted voting often resolves the tie sensibly.

> ⚠️ So **Always scale features** (e.g., standardize to mean 0, std 1).  
> Otherwise the feature with the largest units dominates distance.

**Distance-weighted vote (optional):**  
$w_i=\dfrac{1}{\operatorname{dist}(x,x_i)+\epsilon}$ so closer neighbors count more.

---

## 3) Choosing **k** & common pitfalls

Pick k by validation or cross‑validation: try several values and choose the one with the best validation score.

- **Small k** (1–3): low bias, high variance → can be noisy.
- **Larger k** (5–21+): smoother, higher bias → may underfit.
- Pick **k** via validation or cross-validation.

**Pitfalls**

- Not scaling features.
- Using an unfitting distance (e.g., Euclidean on sparse text).
- Class imbalance: consider distance-weighted vote or class-weighted strategies.

---

## 4) kNN — minimal code (easy to try)

Before running the code below, here’s the flow you’ll see:

- Make a small, separable dataset and split into train/test.
- Fit the scaler **on training data only**, then transform both train and test.
- Loop over a few k values; each model just stores the scaled training set.
- Predict by finding the k nearest training points for each test point and aggregating their labels.

> Tip: run locally. Install: `pip install numpy scikit-learn`

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# 1) Toy 2D dataset
X, y = make_classification(n_samples=400, n_features=2, n_redundant=0,
                           n_informative=2, n_clusters_per_class=1, random_state=7)
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.25, random_state=7)

# 2) Scale for distance-based methods
scaler = StandardScaler().fit(Xtr)
Xtr = scaler.transform(Xtr)
Xte = scaler.transform(Xte)

# 3) Try a few k values (distance-weighted)
for k in [1, 3, 5, 11, 21]:
    clf = KNeighborsClassifier(n_neighbors=k, weights="distance", metric="euclidean")
    clf.fit(Xtr, ytr)
    acc = accuracy_score(yte, clf.predict(Xte))
    print(f"k={k:>2}  acc={acc:.3f}")

**From-scratch (core idea):**

In [ ]:
import numpy as np

def knn_predict(Xtr, ytr, x, k=5):
    d = ((Xtr - x)**2).sum(axis=1)**0.5          # Euclidean
    idx = np.argpartition(d, k)[:k]              # top-k neighbors (unordered)
    votes = np.bincount(ytr[idx])
    return votes.argmax()

---

## 5) K-Means — the core idea (intuition)

Choose **K** cluster centers (centroids) so that points are as close as possible to the centroid of the cluster they’re assigned to. The loss being minimized is the sum of **squared** distances to the nearest centroid, which naturally makes the mean the best representative for each cluster and favors compact, roughly spherical clusters.

> **Goal:** group similar points into **K** clusters.  
> Each cluster has a center (the **mean**), and points belong to their nearest center.

**Plain-English objective:** place $K $ centers so points are as close as possible (on average, squared) to their assigned center.

**Mathematically, with clusters $C_k$ and centroids $\mu_k$:**

$$
\min_{\{C_k\},\{\mu_k\}} \sum_{k=1}^{K} \sum_{x\in C_k} \lVert x-\mu_k\rVert_2^2
$$

and the centroid update

$$
\mu_k \;=\; \frac{1}{\lvert C_k\rvert}\sum_{x\in C_k} x\;.
$$

---

## 6) Lloyd’s algorithm (the standard K-Means loop)

Two repeated steps drive the method:

1. **assignment** — give each point to its nearest centroid.
2. **update** — move each centroid to the mean of its assigned points. Each step never increases the objective, so the loop converges to a local optimum. Initialization matters; using **k‑means++** spreads starting centroids apart and improves results. If a cluster becomes empty, re‑seed that centroid (e.g., pick a faraway point).

Repeat until nothing changes.

Each iteration never increases the objective → **converges** to a local optimum.

**Good practice**

- **k-means++** initialization (smart starting centers)
- **Multiple restarts** (keep the best run)
- **Scale features** before clustering
- Handle empty clusters by re-seeding (e.g., to a farthest point)

---

## 7) Picking **K** (how many clusters?)

- **Elbow method:** plot $K$ vs **inertia** (sum of squared distances to centers).  
  Look for the “bend” where adding clusters gives diminishing returns.
- **Silhouette score** ($-1 \ldots 1$): higher is better. Rough guide:  
  $0.5+$ good separation, $\sim 0.25$ weak structure.

---

## 8) K-Means — minimal code (easy to try)

Scan a few K values, compute inertia and silhouette for each, and then pick a reasonable K. Use `init="k-means++"` and multiple `n_init` runs to avoid poor local optima. After selecting K, fit once more and evaluate.

In [ ]:
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# 1) Synthetic data with 4 clusters
X, _, _ = make_blobs(n_samples=600, centers=4, cluster_std=1.2, random_state=42)
X = StandardScaler().fit_transform(X)

# 2) Try several K to compute inertia and silhouette
Ks = range(2, 9)
inertias, sils = [], []
for K in Ks:
    km = KMeans(n_clusters=K, n_init=10, init="k-means++", random_state=42).fit(X)
    inertias.append(km.inertia_)
    sils.append(silhouette_score(X, km.labels_))

print("K:", list(Ks))
print("Inertia:", [round(v, 1) for v in inertias])
print("Silhouette:", [round(v, 3) for v in sils])

# 3) Pick K (say best-looking), then fit once more
bestK = 4
km = KMeans(n_clusters=bestK, n_init=10, init="k-means++", random_state=42).fit(X)
print("Final silhouette at K=4:", round(silhouette_score(X, km.labels_), 3))

**From-scratch (one iteration):**

In [ ]:
import numpy as np

def kmeans_step(X, centers):
    # Assign
    d = np.stack([np.linalg.norm(X - c, axis=1) for c in centers], axis=1)
    labels = d.argmin(axis=1)
    # Update
    new_centers = np.array([X[labels == k].mean(axis=0) for k in range(centers.shape[0])])
    return labels, new_centers

---

## 9) kNN vs K-Means — quick cheat sheet

Use **kNN** when you have labels and want predictions; use **K‑Means** when you don’t have labels and want segments. kNN has almost no training cost but can be slow at prediction time; K‑Means costs more to fit but predicts fast by checking the nearest centroid. Scaling is important for both.

| Aspect       | kNN (Supervised)                             | K-Means (Unsupervised)                      |
| ------------ | -------------------------------------------- | ------------------------------------------- |
| Purpose      | Predict labels/values from neighbors         | Discover groups without labels              |
| “Training”   | Just store data                              | Learn K centers                             |
| Compute cost | Cheap train, slower predict (need neighbors) | Fast predict, cost during fitting           |
| Key choices  | **k**, distance, weighting, scaling          | **K**, init (k-means++), scaling, restarts  |
| When to use  | Strong local patterns, simple baseline       | Quick segmentation, preprocessing, insights |

---

## 10) Checklists, FAQs, and practice

kNN checklist: scale features; tune **k** with validation; consider distance‑weighted voting; watch for class imbalance.

K‑Means checklist: scale features; use k‑means++; try several K and compare inertia and silhouette; handle empty clusters robustly.

Practice ideas: try kNN on Iris or a small MNIST subset with various k; for K‑Means, sweep K=2…10 and plot inertia and silhouette; as a speed‑up experiment, cluster first, then run kNN within each cluster and compare runtime/accuracy to vanilla kNN.

**kNN checklist**

- [ ] Scale features
- [ ] Pick **k** via validation
- [ ] Consider distance-weighted vote
- [ ] Beware class imbalance

**K-Means checklist**

- [ ] Scale features
- [ ] Use **k-means++** + multiple **n_init**
- [ ] Pick **K** via elbow/silhouette
- [ ] Watch for empty clusters (reseed)

**FAQs**

- _“kNN is slow at prediction.”_ → Use KD-Tree/Ball-Tree/ANN, or pre-cluster and search within cluster.
- _“My clusters look weird.”_ → Scale features; try different K; check for non-spherical structure (K-Means likes spherical blobs).
- _“Do I need labels for K-Means?”_ → No — it’s unsupervised.

**Practice**

1. On a real dataset (e.g., Iris or a MNIST subset), tune **k** for kNN and report accuracy.
2. Run K-Means with $K = 2,\dots,10$; plot inertia and silhouette; choose $K$ and visualize clusters.
3. Hybrid: cluster first (K-Means), then run kNN **within each cluster** — compare speed/accuracy.

---

## Summary

kNN: predict using the answers from your **k nearest labeled neighbors**.

K‑Means: place **K centroids** and alternate between assigning points to the nearest centroid and recomputing centroids until stable.

You learned:

- **kNN**: how neighbors + distances make predictions; how **k** and scaling affect results.
- **K-Means**: how Lloyd’s algorithm works; how to pick **K** and get stable clusters.
- Minimal code to try both methods today.

**Next:** Support vector machines